In [58]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [59]:
cd drive/MyDrive/csc413/

[Errno 2] No such file or directory: 'drive/MyDrive/csc413/'
/content/drive/MyDrive/csc413


In [60]:
!pip install mtcnn

In [61]:
import matplotlib.pyplot as plt
import numpy as np
import torch, os
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import argparse
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, utils
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from util import *
from main import *

In [62]:
classes = {
    0: 'Surprise', 1: 'Fear', 2: 'Disgust', 3: 'Happy', 4: 'Sad', 5: 'Anger', 6: 'Neutral'
}

In [63]:
def conv_block(in_chnl, out_chnl, pool=False, padding=1):
    layers = []
    layers.append(nn.Conv2d(in_chnl, out_chnl, kernel_size=3, padding=padding))
    layers.append(nn.BatchNorm2d(out_chnl))
    layers.append(nn.ReLU(inplace=True))
    if pool: layers.append(nn.MaxPool2d(2))
    return nn.Sequential(*layers)

In [64]:
class Base(nn.Module):
    def training_step(self, batch):
        images, labels = batch
        out = self(images)
        loss = F.cross_entropy(out, labels)
        return loss
    
    def validating(self, batch):
        images, labels = batch
        out = self(images)
        loss = F.cross_entropy(out, labels)
        acc = accuracy(out, labels)
        return {'val_loss': loss.detach(), 'val_acc': acc}

In [65]:
class ResNet(Base):
    def __init__(self, in_chnls, num_cls):
        super().__init__()
        self.conv1 = conv_block(in_chnls, 64, pool=True)
        self.conv2 = conv_block(64, 128, pool=True)
        self.resnet1 = nn.Sequential(conv_block(128, 128), conv_block(128, 128))
        self.conv3 = conv_block(128, 256, pool=True)
        self.conv4 = conv_block(256, 512, pool=True)
        self.resnet2 = nn.Sequential(conv_block(512, 512), conv_block(512, 512))
        self.classifier = nn.Sequential(nn.MaxPool2d(3), nn.Flatten(), nn.Linear(512, num_cls))
        
    def forward(self, xb):
        out = self.conv1(xb)
        out = self.conv2(out)
        out = self.resnet1(out) + out
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.resnet2(out) + out
        return self.classifier(out)

In [66]:
class VGG(Base):
    def __init__(self, in_chnls, num_cls):
        super(VGG, self).__init__()
        vgg = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M']
        layers = []
        in_channels = in_chnls
        for x in vgg:
            if x == 'M':
                layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
            else:
                layers.append(nn.Conv2d(in_channels, x, kernel_size=3, padding=1))
                layers.append(nn.BatchNorm2d(x))
                layers.append(nn.ReLU(inplace=True))
                in_channels = x
        layers.append(nn.AvgPool2d(kernel_size=1, stride=1))
        self.features = nn.Sequential(*layers)
        self.classifier = nn.Linear(512, num_cls)
        
    def forward(self, x):
        out = self.features(x)
        out = out.view(out.size(0), -1)
        out = F.dropout(out, p=0.5, training=self.training)
        out = self.classifier(out)
        return out

In [67]:
def get_default_device():
    if torch.cuda.is_available(): return torch.device('cuda')
    else: return torch.device('cpu')

In [68]:
class Dataset(Dataset):
    def __init__(self, images, labels, transforms):
        self.inputs = images
        self.labels = labels
        self.transforms = transforms
    
    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, i):
        data = self.inputs[i]
        data = np.asarray(data).astype(np.uint8).reshape(48,48,1)
        data = self.transforms(data)
        label = self.labels[i]
        return (data, label)

In [69]:
class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl:
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)
        

In [70]:
def evaluate(model, val_loader):
    model.eval()
    #print(model)
    outputs = [model.validating(batch) for batch in val_loader]
    batch_losses = [x['val_loss'] for x in outputs]
    epoch_loss = torch.stack(batch_losses).mean()
    batch_accs = [x['val_acc'] for x in outputs]
    epoch_acc = torch.stack(batch_accs).mean()
    return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

In [71]:
def fit(epochs, max_lr, model, train_loader, val_loader, weight_decay, grad_clip, opt_func=torch.optim.Adam):
    torch.cuda.empty_cache()
    history = []
    optimizer = opt_func(model.parameters(), max_lr, weight_decay=weight_decay)
    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epochs, steps_per_epoch=len(train_loader))
    for epoch in range(epochs):
        model.train()
        train_losses = []
        lrs = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            if grad_clip: nn.utils.clip_grad_value_(model.parameters(), grad_clip)
            optimizer.step()
            optimizer.zero_grad()
            lrs.append(get_lr(optimizer))
            sched.step()
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        result['lrs'] = lrs
        print("Epoch [{}], last_lr: {:.5f}, train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['lrs'][-1], result['train_loss'], result['val_loss'], result['val_acc']))
        history.append(result)
    return history

In [72]:
def plot_lrs(history):
    lrs = np.concatenate([x.get('lrs', []) for x in history])
    plt.figure()
    plt.plot(lrs)
    plt.xlabel('Batch no.')
    plt.ylabel('Learning rate')
    plt.title('Learning Rate vs. Batch no.')
    plt.legend()
    plt.show()

In [75]:
def testBias(t, name):
    data = np.load("../data/test.npz")
    model = None
    if t=="VGG":
        print("VGG")
        model = VGG(1, 7)
    else:
        model = ResNet(1, 7)
    
    model.load_state_dict(torch.load(name + ".pth", map_location=get_default_device()))
    model.cuda()
    res = []

    valid_transform = transforms.Compose(
    [
        transforms.ToPILImage(),
        transforms.Grayscale(num_output_channels=1),
        transforms.ToTensor(),
        transforms.Normalize((0.5), (0.5))
    ])
    batch_num = 120

    for i in range(7):
        test_images = data["test_images"+str(i)]
        test_labels = data["test_labels"+str(i)]
        valid_data = Dataset(test_images, test_labels, valid_transform)
        validDataLoader = DataLoader(valid_data, batch_num*2, num_workers=4, pin_memory=True)
        device = get_default_device()
        validDataLoader = DeviceDataLoader(validDataLoader, device)
        result = evaluate(model, validDataLoader)
        print("Emotion [{}], val_loss: {:.4f}, val_acc: {:.4f}".format(
            i, result['val_loss'], result['val_acc']))
        res.append(result['val_acc'])

    return res


In [ ]:
if __name__ == "__main__":
    # New dataset with VGG model
    data = np.load("../data/new_data.npz")
    print("Load the data file successfully, start training:")
    model_type = "VGG"
    train_images = data["train_images"]
    train_labels = data["train_labels"]
    test_images = data["test_images"]
    test_labels = data["test_labels"]

    train_transform = transforms.Compose(
    [
        transforms.ToPILImage(),
        transforms.Grayscale(num_output_channels=1),
        transforms.RandomCrop(48, padding=4, padding_mode='reflect'),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5), (0.5), inplace=True)
    ])

    valid_transform = transforms.Compose(
    [
        transforms.ToPILImage(),
        transforms.Grayscale(num_output_channels=1),
        transforms.ToTensor(),
        transforms.Normalize((0.5), (0.5))
    ])

    train_data = Dataset(train_images, train_labels, train_transform)
    valid_data = Dataset(test_images, test_labels, valid_transform)
    print("Training and validation dataset initialization finished")
    torch.manual_seed(19)
    batch_num = 120
    trainDataLoader = DataLoader(train_data, batch_num, shuffle=True, num_workers=4, pin_memory=True)
    validDataLoader = DataLoader(valid_data, batch_num*2, num_workers=4, pin_memory=True)
    print("TrainDataLoader and validDataLoader initialization finished")
    device = get_default_device()
    trainDataLoader = DeviceDataLoader(trainDataLoader, device)
    validDataLoader = DeviceDataLoader(validDataLoader, device)

    if model_type=="VGG":
        model = to_device(VGG(1, 7), device)
    else:
        model = to_device(ResNet(1, 7), device)
    print("Get the model type:" , model_type)
    print(model)
    print("Start model evaluation:")
    evaluate(model, validDataLoader)
    max_lr = 0.001
    grad_clip = 0.1
    weight_decay = 1e-4
    print("Start model fitting:")
    trainLog = fit(60, max_lr, model, trainDataLoader, validDataLoader, weight_decay, grad_clip, torch.optim.Adam)
    torch.save(model.state_dict(), "VGG_new"+'.pth')
    plot_losses(trainLog)
    plot_lrs(trainLog)

In [ ]:
    # Old dataset with VGG model
    data = np.load("../data/data.npz")
    print("Load the data file successfully, start training:")
    model_type = "VGG"
    train_images = data["train_images"]
    train_labels = data["train_labels"]
    test_images = data["test_images"]
    test_labels = data["test_labels"]

    train_transform = transforms.Compose(
    [
        transforms.ToPILImage(),
        transforms.Grayscale(num_output_channels=1),
        transforms.RandomCrop(48, padding=4, padding_mode='reflect'),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5), (0.5), inplace=True)
    ])

    valid_transform = transforms.Compose(
    [
        transforms.ToPILImage(),
        transforms.Grayscale(num_output_channels=1),
        transforms.ToTensor(),
        transforms.Normalize((0.5), (0.5))
    ])

    train_data = Dataset(train_images, train_labels, train_transform)
    valid_data = Dataset(test_images, test_labels, valid_transform)
    print("Training and validation dataset initialization finished")
    torch.manual_seed(19)
    batch_num = 120
    trainDataLoader = DataLoader(train_data, batch_num, shuffle=True, num_workers=4, pin_memory=True)
    validDataLoader = DataLoader(valid_data, batch_num*2, num_workers=4, pin_memory=True)
    print("TrainDataLoader and validDataLoader initialization finished")
    device = get_default_device()
    trainDataLoader = DeviceDataLoader(trainDataLoader, device)
    validDataLoader = DeviceDataLoader(validDataLoader, device)

    if model_type=="VGG":
        model = to_device(VGG(1, 7), device)
    else:
        model = to_device(ResNet(1, 7), device)
    print("Get the model type:" , model_type)
    print(model)
    print("Start model evaluation:")
    evaluate(model, validDataLoader)
    max_lr = 0.001
    grad_clip = 0.1
    weight_decay = 1e-4
    print("Start model fitting:")
    trainLog = fit(60, max_lr, model, trainDataLoader, validDataLoader, weight_decay, grad_clip, torch.optim.Adam)
    torch.save(model.state_dict(), "VGG"+'.pth')
    plot_losses(trainLog)
    plot_lrs(trainLog)

In [ ]:
    # New dataset with ResNet model.
    data = np.load("../data/new_data.npz")
    print("Load the data file successfully, start training:")
    model_type = "ResNet"
    train_images = data["train_images"]
    train_labels = data["train_labels"]
    test_images = data["test_images"]
    test_labels = data["test_labels"]

    train_transform = transforms.Compose(
    [
        transforms.ToPILImage(),
        transforms.Grayscale(num_output_channels=1),
        transforms.RandomCrop(48, padding=4, padding_mode='reflect'),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5), (0.5), inplace=True)
    ])

    valid_transform = transforms.Compose(
    [
        transforms.ToPILImage(),
        transforms.Grayscale(num_output_channels=1),
        transforms.ToTensor(),
        transforms.Normalize((0.5), (0.5))
    ])

    train_data = Dataset(train_images, train_labels, train_transform)
    valid_data = Dataset(test_images, test_labels, valid_transform)
    print("Training and validation dataset initialization finished")
    torch.manual_seed(19)
    batch_num = 120
    trainDataLoader = DataLoader(train_data, batch_num, shuffle=True, num_workers=4, pin_memory=True)
    validDataLoader = DataLoader(valid_data, batch_num*2, num_workers=4, pin_memory=True)
    print("TrainDataLoader and validDataLoader initialization finished")
    device = get_default_device()
    trainDataLoader = DeviceDataLoader(trainDataLoader, device)
    validDataLoader = DeviceDataLoader(validDataLoader, device)

    if model_type=="VGG":
        model = to_device(VGG(1, 7), device)
    else:
        model = to_device(ResNet(1, 7), device)
    print("Get the model type:" , model_type)
    print(model)
    print("Start model evaluation:")
    evaluate(model, validDataLoader)
    max_lr = 0.001
    grad_clip = 0.1
    weight_decay = 1e-4
    print("Start model fitting:")
    trainLog = fit(6, max_lr, model, trainDataLoader, validDataLoader, weight_decay, grad_clip, torch.optim.Adam)
    torch.save(model.state_dict(), 'ResNet_new'+'.pth')
    plot_losses(trainLog)
    plot_lrs(trainLog)

In [ ]:
    # Old dataset with ResNet model
    data = np.load("../data/data.npz")
    print("Load the data file successfully, start training:")
    model_type = "ResNet"
    train_images = data["train_images"]
    train_labels = data["train_labels"]
    test_images = data["test_images"]
    test_labels = data["test_labels"]

    train_transform = transforms.Compose(
    [
        transforms.ToPILImage(),
        transforms.Grayscale(num_output_channels=1),
        transforms.RandomCrop(48, padding=4, padding_mode='reflect'),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5), (0.5), inplace=True)
    ])

    valid_transform = transforms.Compose(
    [
        transforms.ToPILImage(),
        transforms.Grayscale(num_output_channels=1),
        transforms.ToTensor(),
        transforms.Normalize((0.5), (0.5))
    ])

    train_data = Dataset(train_images, train_labels, train_transform)
    valid_data = Dataset(test_images, test_labels, valid_transform)
    print("Training and validation dataset initialization finished")
    torch.manual_seed(19)
    batch_num = 120
    trainDataLoader = DataLoader(train_data, batch_num, shuffle=True, num_workers=4, pin_memory=True)
    validDataLoader = DataLoader(valid_data, batch_num*2, num_workers=4, pin_memory=True)
    print("TrainDataLoader and validDataLoader initialization finished")
    device = get_default_device()
    trainDataLoader = DeviceDataLoader(trainDataLoader, device)
    validDataLoader = DeviceDataLoader(validDataLoader, device)

    if model_type=="VGG":
        model = to_device(VGG(1, 7), device)
    else:
        model = to_device(ResNet(1, 7), device)
    print("Get the model type:" , model_type)
    print(model)
    print("Start model evaluation:")
    evaluate(model, validDataLoader)
    max_lr = 0.001
    grad_clip = 0.1
    weight_decay = 1e-4
    print("Start model fitting:")
    trainLog = fit(40, max_lr, model, trainDataLoader, validDataLoader, weight_decay, grad_clip, torch.optim.Adam)
    torch.save(model.state_dict(), 'ResNet'+'.pth')
    plot_losses(trainLog)
    plot_lrs(trainLog)

In [93]:
    t = "ResNet"
    res = testBias(t, t)
    print(res)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Emotion [0], val_loss: 0.5258, val_acc: 0.8587
Emotion [1], val_loss: 5.6634, val_acc: 0.3103
Emotion [2], val_loss: 3.2309, val_acc: 0.4709
Emotion [3], val_loss: 0.3075, val_acc: 0.9158
Emotion [4], val_loss: 0.9783, val_acc: 0.7331
Emotion [5], val_loss: 1.2413, val_acc: 0.7191
Emotion [6], val_loss: 0.1838, val_acc: 0.9214
[0.8587499856948853, 0.3103448152542114, 0.47089946269989014, 0.9158332943916321, 0.7331270575523376, 0.7191011309623718, 0.9213973879814148]


In [94]:
    t = "ResNet"
    name = "ResNet_new"
    res = testBias(t, name)
    print(res)    

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Emotion [0], val_loss: 0.5604, val_acc: 0.8846
Emotion [1], val_loss: 5.7282, val_acc: 0.3362
Emotion [2], val_loss: 3.9904, val_acc: 0.4709
Emotion [3], val_loss: 1.0110, val_acc: 0.7658
Emotion [4], val_loss: 1.1366, val_acc: 0.7438
Emotion [5], val_loss: 1.2221, val_acc: 0.7528
Emotion [6], val_loss: 0.2845, val_acc: 0.9214
[0.8845833539962769, 0.3362068831920624, 0.47089946269989014, 0.76583331823349, 0.743811845779419, 0.7528089880943298, 0.9213973879814148]


In [95]:
t = "VGG"
res = testBias(t, t)
print(res)

VGG


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Emotion [0], val_loss: 0.7832, val_acc: 0.8467
Emotion [1], val_loss: 6.1306, val_acc: 0.3448
Emotion [2], val_loss: 3.7729, val_acc: 0.4709
Emotion [3], val_loss: 0.4480, val_acc: 0.9075
Emotion [4], val_loss: 1.0825, val_acc: 0.7798
Emotion [5], val_loss: 1.1203, val_acc: 0.7921
Emotion [6], val_loss: 0.3617, val_acc: 0.9170
[0.846666693687439, 0.3448275923728943, 0.47089946269989014, 0.9074999690055847, 0.7797648310661316, 0.7921348214149475, 0.9170305728912354]


In [97]:
t = "VGG"
name = "VGG_new"
res = testBias(t, name)
print(res)

VGG


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Emotion [0], val_loss: 0.5033, val_acc: 0.8987
Emotion [1], val_loss: 5.6059, val_acc: 0.3448
Emotion [2], val_loss: 3.8129, val_acc: 0.4868
Emotion [3], val_loss: 1.6241, val_acc: 0.7683
Emotion [4], val_loss: 1.1567, val_acc: 0.7998
Emotion [5], val_loss: 1.2320, val_acc: 0.7921
Emotion [6], val_loss: 0.3220, val_acc: 0.9258
[0.8987499475479126, 0.3448275923728943, 0.48677247762680054, 0.7683333158493042, 0.7998143434524536, 0.7921348214149475, 0.9257642030715942]
